<a href="https://colab.research.google.com/github/Lukehsu1999/Academic-and-Formality-Rewriter/blob/main/model/BERT_Formality_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Installation</h1>

In [ ]:
!pip install -q transformers

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
try:
  import GPUtil as GPU
  GPUs = GPU.getGPUs()
  device='/gpu:0'
except:
  device='/cpu:0'

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from transformers import TFBertForSequenceClassification
from transformers import BertTokenizer

<h1>Preparing Data</h1>

put the path to the GYAFC below once you have gained access to the corpus

In [ ]:
# these are academic-style

path_to_formal_train_file="/content/drive/MyDrive/CityU/FYP/FormalSet/EMFR_formal_train"
path_to_informal_train_file="/content/drive/MyDrive/CityU/FYP/FormalSet/EMFR_informal_train"

path_to_formal_val_file = "/content/drive/MyDrive/CityU/FYP/FormalSet/EMFR_formal_eval"
path_to_informal_val_file = "/content/drive/MyDrive/CityU/FYP/FormalSet/EMFR_informal_eval"

# but these are
path_to_formal_test_file="/content/drive/MyDrive/CityU/FYP/FormalSet/EMFR_formal_test"
path_to_informal_test_file="/content/drive/MyDrive/CityU/FYP/FormalSet/EMFR_informal_test"

Define how many lines from formal set you take (same num as from informal set).<br/>
Total take=2*take_formal

In [ ]:
#feel free to change
#train_take_formal=1200
#valid_take_formal=400 #validation set is useless if you don't run more than one epoch
num_epochs= 1
batch_size = 16
period_less=False

Train set

In [ ]:
trainformal=open(path_to_formal_train_file,"r")
trainflines=trainformal.readlines()

ds_train=[]

#label: 1: formal; 0: informal

for line in trainflines:

  #only formal set have period concerns
  if period_less:
    line=line[:-2]

  ds_train.append({'label':1,'text':line})

traininformal=open(path_to_informal_train_file,"r")
traininflines=traininformal.readlines()

for line in traininflines:

  ds_train.append({'label':0,'text':line})

print(len(ds_train))
if period_less:
  print("period_less")
else:
  print("include period")


190112
include period


Validation Set

In [ ]:
valformal=open(path_to_formal_val_file,"r")
valflines=valformal.readlines()

ds_valid=[]

#label: 1: formal; 0: informal

for line in valflines:

  #only formal set have period concerns
  if period_less:
    line=line[:-2]

  ds_valid.append({'label':1,'text':line})

valinformal=open(path_to_informal_val_file,"r")
valinflines=valinformal.readlines()

for line in valinflines:


  ds_valid.append({'label':0,'text':line})

print(len(ds_valid))

if period_less:
  print("period_less")
else:
  print("include period")

19012
include period


GYAFC Given Test set (unchangeable, use as an performance comparison between different size of models)

In [ ]:
testformal=open(path_to_formal_test_file,"r")
testflines=testformal.readlines()
print("test formal lines cnt:",len(testflines))
ds_test=[]

#label: 1: formal; 0: informal

for line in testflines:
  ds_test.append({'label':1,'text':line})

testinformal=open(path_to_informal_test_file,"r")
testinflines=testinformal.readlines()
print("test informal lines cnt:",len(testinflines))

for line in testinflines:
  ds_test.append({'label':0,'text':line})

print(len(ds_test))

test formal lines cnt: 2101
test informal lines cnt: 2748
4849


<h1>Data Preprocessing</h1>

In [ ]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# can be up to 512 for BERT
max_length = 128

In [ ]:
def convert_example_to_feature(review):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
  
  return tokenizer.encode_plus(review, 
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )

In [ ]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

In [ ]:
def encode_examples(ds, limit=-1):

  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  if (limit > 0):
      ds = ds.take(limit)
    
  for i in range(0,len(ds)): 

    bert_input = convert_example_to_feature(ds[i]["text"])
  
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append(ds[i]["label"])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

<h2>Encode the data<h2>

Train and validation

In [ ]:
ds_train_encoded=encode_examples(ds_train).shuffle(len(ds_train)).batch(batch_size)
ds_valid_encoded=encode_examples(ds_valid).shuffle(len(ds_train)).batch(batch_size)


Test

In [ ]:
ds_test_encoded = encode_examples(ds_test).batch(batch_size)
#ds_10Ktest_encoded=encode_examples(ds_10Ktest).batch(batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#length=lines/batch_size
print(len(ds_train_encoded))
print(len(ds_valid_encoded))
print(len(ds_test_encoded))
#print(len(ds_10Ktest_encoded))

11882
1189
304


<h1>Model Compile & Train</h1>

In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5

number_of_epochs = num_epochs #change 1 to 3



# model initialization: building a model from scratch
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')
# if load weights from previous checkpoints
path_to_model_weight = "/content/drive/MyDrive/CityU/FYP/FormalSet/Epoch2Jun15" 
model.load_weights(path_to_model_weight)

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_valid_encoded)

In [ ]:
#need to have more than 1 epoch to show the curve
print(bert_history.history.keys())
print(bert_history)
# summarize history for accuracy
from matplotlib import pyplot as plt
plt.plot(bert_history.history['accuracy'])
plt.plot(bert_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(bert_history.history['loss'])
plt.plot(bert_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save_weights("/content/drive/MyDrive/CityU/FYP/FormalSet/Epoch3Jun15")

In [ ]:
model.evaluate(ds_test_encoded)

304/304 [==============================] - 45s 147ms/step - loss: 0.1435 - accuracy: 0.9429


[0.1434587836265564, 0.9428748488426208]

<h1>Evaluate with Test </h1>

put the path to your model below

In [ ]:
path_to_model="/content/drive/MyDrive/CityU/FYP/Mar29Split/test/AcademicEpoch1June12"

In [ ]:
#import model
model=TFBertForSequenceClassification.from_pretrained('bert-base-cased')
#paste the path to the saved weights
model.load_weights(path_to_model)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
learning_rate = 2e-5

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

For Evaluating Models

In [ ]:
model.evaluate(ds_test_encoded)

304/304 [==============================] - 47s 145ms/step - loss: 0.2853 - accuracy: 0.9091


[0.28527092933654785, 0.909053385257721]

In [ ]:
#model.evaluate(ds_10Ktest_encoded)

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


<h1>Tryouts</h1>

In [ ]:
!pip install -q transformers

In [ ]:
import tensorflow as tf
from tensorflow import keras
from transformers import TFBertForSequenceClassification
from transformers import BertTokenizer

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
def getPrediction(sentence,thismodel):
  inputs=tokenizer(sentence,return_tensors="tf")
  outputs=thismodel(inputs,return_dict=True)
  #print(outputs)
  #print(outputs.loss)
  #print(outputs.logits)
  logits=outputs.logits
  probs = tf.nn.softmax(logits)
  prediction = tf.math.argmax(probs, axis=1)
  print(f"probs {probs}")
  if prediction[0]==1:
    print("Formal")
    return 1
  else:
    print("Informal")
    return 0


Please go to https://drive.google.com/drive/folders/1mDlYfLHuIiNA8kHZooXAqpk5eZBOj39o?usp=sharing to download the model weights
and paste the path in the path_to_model_weight below

In [ ]:
path_to_model_weight="/content/drive/MyDrive/CityU/FYP/Bert_Formality_Classifier/Feb21test"
demo_model=TFBertForSequenceClassification.from_pretrained('bert-base-cased')
demo_model.load_weights(path_to_model_weight)

In [ ]:
#put your sentence in the input_sentence
input_sentence="I dunno wat happening here."

In [ ]:
getPrediction(input_sentence,demo_model) #demo_model

<h1> Saving model weights <h2>

In [ ]:
model.save_weights("")

Insight: adding "." at the end of the sentence makes it too much more formal. Need to rethink about it